{{title_s1_4}}

This notebook demonstrates how to access Sentinel-1 RTC imagery from [Microsoft Planetary Computer](https://planetarycomputer.microsoft.com/) using [`stackstac`](https://stackstac.readthedocs.io/en/latest/). [STAC](https://stacspec.org/en) stands for Spatio-Temporal Asset Catalog, it is a common framework to describe geospatial information and a way for data providers, developers, and users to work and exchange information efficiently. You can read more about STAC [here](https://stacspec.org/en) and checkout more useful [tutorials](https://stacspec.org/en/tutorials/) for working with STAC data.

::::{tab-set}
:::{tab-item} Outline  

(content.Section_A)=
**[A. Connect to Microsoft Planetary Computer](#a-connect-to-microsoft-planetary-computer)**
- {{a1_s1_nb4}}

(content.Section_B)=
**[B. Read data and create Xarray data cube](#b-read-data-with-xarray)**
- {{b1_s1_nb4}}
- {{b2_s1_nb4}}
- {{b3_s1_nb4}}

(content.Section_C)= 
**[C. Visualize data](#c-visualize-data)**
- {{c1_s1_nb4}}
- {{c2_s1_nb4}}
- {{c3_s1_nb4}}

:::
:::{tab-item} Learning goals

{{concepts}}
- Querying large cloud-hosted dataset
- Accessing cloud-hosted data stored as COGs (cloud-optimized GeoTIFFs)
- Extracting and organizing metadata

{{techniques}}
- Introduction to working with STAC data
- Using `pystac_client` to query cloud-hosted datasets, observe metadata
- Using `stackstac` to read cloud-hosted data as xarray objects
- Using `xarray` to manipulate and organize Sentinel-1 SAR data
- Performing grouping and reductions on `xarrray` objects
- Visualizing `xarray` objects using `FacetGrid`
:::
::::

In [ ]:
%xmode minimal
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import planetary_computer
from pystac_client import Client
import rich.table
import stackstac

import s1_tools

{{break}}

## A. Connect to Microsoft Planetary Computer 

We use the [`pystac_client`](https://pystac-client.readthedocs.io/) package to interact with and query the Microsoft Planetary Computer [Sentinel-1 RTC dataset](https://planetarycomputer.microsoft.com/dataset/group/sentinel-1). In the cell below, we will create an object called `catalog` by calling the `.open()` method of the `Client` class. This establishes a connection with the data hosted at the url provided.

### {{a1_s1_nb4}}

In [ ]:
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
catalog

Next, define some parameters to help us query the data catalog for a specific collection, time range, and geographic area of interest.

In [ ]:
time_range = "2021-05-04/2022-05-21"
bbox = [88.214935, 27.92767, 88.302, 28.034]

Now, search the catalog for entries that match the specified criteria for collection (Sentinel-1 RTC), bbox (the AOI defined above) and datetime (the specified time range):

In [ ]:
search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox, datetime=time_range)
items = search.item_collection()
len(items)

We've created a few more instances of `pystac_client` classes. Check out the object types below to better familiarize yourself with the STAC metadata objects.

In [ ]:
print(type(catalog))
print(type(search))
print(type(items))

`items` is an instance of the class `ItemCollection`; we can explore it via the embedded html interface.

In [ ]:
items

To make it easier to work with, we can convert  the `items` object to a dictionary, and from there, to a `geopandas.GeoDataFrame`. The metadata from within each `item` of the `ItemCollection` object is present in the `GeoDataFrame` but its easier to scan and organize this way. 

In [ ]:
df = gpd.GeoDataFrame.from_features(items.to_dict(), crs="epsg:4326")
df.head(5)

Construct a table with metadata for a single scene (ie. a single element of the list `items`). This is similar (but not identical) to the information stored in the file name and README files of the Sentinel-1 data we read from local storage.

In [ ]:
table = rich.table.Table("key", "value")
for k, v in sorted(items[0].properties.items()):
    table.add_row(k, str(v))
table

We can also explore the object metadata outside of the table. Try typing `.assets`, `.links`, `.STAC_extensions` and `.properties` onto the term below. 
You can query the object programmatically for the same metadata stored in the table using dictionary syntax on the `properties` accessor.

In [ ]:
items[0]
# items[0].assets
# items[0].links
# items[0].properties

Now that we'e explored the items that fit our query of the dataset and seen the metadata, let's read the data using Xarray. 

## B. Read data with Xarray

### {{b1_s1_nb4}}

We use [`dask.distributed`](https://distributed.dask.org/en/stable/) to parallelize operations in this notebook. Similarly to PySTAC, Dask distributed also uses a client. The dask distributed client allows us to interact with the scheduler that manages jobs across a cluster of compute resources whether they are cores on a local machine or virtual machines distributed across a cloud-compute platform. 

In [ ]:
# give dask client a name other than 'Client'
from dask.distributed import Client as da_Client

client = da_Client(processes=False)
print(client.dashboard_link)

The `client.dashboard_link` points to a dask dashboard for the client we've just initialized. 

### {{b2_s1_nb4}}

Now that we have queried the data that is available from Microsoft Planetary Computer and inspected the metadata using `pystac`, we use `stackstac` to read the data into memory as an Xarray object. Calling `stackstac.stack()` takes a STAC `collection` and produces a lazy `xarray.DataArray` backed by Dask arrays. For more on Dask arrays see [relevant concepts](../../background/relevant_concepts.md).

In [ ]:
type(items)

#### *More detail on `stackstack.stac()`*

In the code cell below, you can see that we pass the object `items`, a `pystac.ItemCollection` to `stackstac.stack()`. The wrapper `planetary_computer.sign()` uses Planetary Computer subscription key credentials to access the data. 

`stackstac` passes the metadata from the STAC collection into the Xarray object as coordinates allowing you to further organize and manipulate the object to fit your purposes. `stackstac` can also read the data in according to parameters passed during the `stack()` call. In the code cell below we pass parameters for bounding box and coordinate reference system. To specify the resolution as something other than the resolution at which its stored, pass a `resolution = ` argument. 

In [ ]:
da = stackstac.stack(
    planetary_computer.sign(items),
    bounds_latlon=bbox,
    epsg=32645,
)

In [ ]:
da = da.persist()

In [ ]:
da = da.compute()

### {{b3_s1_nb4}}

Let's take a look at what `stackstac.stack()` returns.

We have a `xr.DataArray`, meaning there is only one data variable. It has 'x','y','time' dimensions as well as a 'band' dimension. This is how the VV and VH polarization images are stored. Note that this dataset doesn't come with a layover-shadow map like the ASF processed Sentinel-1 RTC dataset. 

In addition, there are 39 coordinate variables that provide information about the backscatter imagery and how it was collected and processed. In addition, the object has `attrs` that describe the spatial resolution and coordinate reference information. 

## C. Visualize data

Let's do a little bit of looking around. We'll define a function to convert the backscatter pixel values from power to dB scale but we won't use it yet. This transformation applies a logarithmic scale to the data which makes visualization easier but we do not want to run any summary statistics on the dB data as it will be distorted.

:::{admonition} A note on visualizing SAR data
The measurements that we're provided in the RTC dataset are in intensity, or power, scale. Often, to visualize SAR backscatter, the data is converted from power to normalized radar cross section (the backscatter coefficient). This is in decibel (dB) units, meaning a log transform has been applied. This transformation makes it easier to visualize variability but it is important not to calculate summary statistics on log-transformed data as it will be distorted. You can read more about these concepts [here](https://hyp3-docs.asf.alaska.edu/guides/introduction_to_sar/#sar-scale).
:::

In [ ]:
s1_tools.power_to_db(da.mean(dim="time")).plot(col="band", cmap=plt.cm.Greys_r);

### {{c1_s1_nb4}}

What if we wanted to look only at imagery taken during `ascending` or `descending` passes of the satellite? Because orbital direction is a non-dimensional coordinate, we need to use `.where()` instead of `.sel()` to subset the dataset according to orbital direction.

For more discussion on orbital direction and the differences between ascending and descending passes, see the previous [notebook](content:orbital_dir_section)

In [ ]:
da_asc = da.where(da["sat:orbit_state"] == "ascending", drop=True)
da_desc = da.where(da["sat:orbit_state"] == "descending", drop=True)

Plot the mean over time of ascending and descending pass scenes:

In [ ]:
asc_min = np.array(
    [
        s1_tools.power_to_db(da_asc.sel(band="vv").mean(dim="time")).min(),
        s1_tools.power_to_db(da_asc.sel(band="vh").mean(dim="time")).min(),
    ]
).min()

asc_max = np.array(
    [
        s1_tools.power_to_db(da_asc.sel(band="vv").mean(dim="time")).max(),
        s1_tools.power_to_db(da_asc.sel(band="vh").mean(dim="time")).max(),
    ]
).max()

fig, axs = plt.subplots(ncols=2, figsize=(15, 7))
s1_tools.power_to_db(da_asc.sel(band="vv").mean(dim="time")).plot(
    cmap=plt.cm.Greys_r,
    ax=axs[0],
    cbar_kwargs=({"label": "dB"}),
    vmin=asc_min,
    vmax=asc_max,
)
s1_tools.power_to_db(da_asc.sel(band="vh").mean(dim="time")).plot(
    cmap=plt.cm.Greys_r, ax=axs[1], cbar_kwargs=({"label": "dB"})
)
fig.suptitle("Mean over time of ascending scenes")
axs[0].set_title("VV")
axs[1].set_title("VH")
for i in range(len(axs)):
    axs[i].tick_params(axis="x", rotation=45)

In [ ]:
desc_min = np.array(
    [
        s1_tools.power_to_db(da_asc.sel(band="vv").mean(dim="time")).min(),
        s1_tools.power_to_db(da_asc.sel(band="vh").mean(dim="time")).min(),
    ]
).min()

desc_max = np.array(
    [
        s1_tools.power_to_db(da_asc.sel(band="vv").mean(dim="time")).max(),
        s1_tools.power_to_db(da_asc.sel(band="vh").mean(dim="time")).max(),
    ]
).max()

fig, axs = plt.subplots(ncols=2, figsize=(15, 7))
s1_tools.power_to_db(da_desc.sel(band="vv").mean(dim="time")).plot(
    cmap=plt.cm.Greys_r,
    ax=axs[0],
    cbar_kwargs=({"label": "dB"}),
    vmin=desc_min,
    vmax=desc_max,
)
s1_tools.power_to_db(da_desc.sel(band="vh").mean(dim="time")).plot(
    cmap=plt.cm.Greys_r,
    ax=axs[1],
    cbar_kwargs=({"label": "dB"}),
    vmin=desc_min,
    vmax=desc_max,
)
fig.suptitle("Mean over time of descending scenes")
axs[0].set_title("VV")
axs[1].set_title("VH")
for i in range(len(axs)):
    axs[i].tick_params(axis="x", rotation=45)

It looks like there is some interesting variability between the two images. What if we wanted to see how these differences persist over time?

### {{c2_s1_nb4}}

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

ax.set_title("Mean backscatter for VH band (red) and VV band (blue) over time")
s1_tools.power_to_db(da.sel(band="vv").mean(dim=["x", "y"])).plot(
    ax=ax, linestyle="None", marker="o", color="blue", label="VV"
)
s1_tools.power_to_db(da.sel(band="vh").mean(dim=["x", "y"])).plot(
    ax=ax, linestyle="None", marker="o", color="red", label="VH"
)
fig.legend(loc="center right")
fig.suptitle("Mean backscatter of VH and VV bands over time, ASC scenes", fontsize=12, y=0.98)
ax.set_title(None)
ax.set_ylabel("dB");

Interesting! It looks like there is more variability in the VH band than the VV band. Over the year, there is about 4 dB variability in the VV band but over twice as much in the VH band. Chapter 2 of the [SAR handbook](https://gis1.servirglobal.net/TrainingMaterials/SAR/Chp2Content.pdf) contains information about how polarization impacts radar returns. 

### 3) Seasonal variability

Next, let's take a look at how backscatter values vary seasonally. To do this we will use xarray's [groupby()]() and [.facetgrid]() methods.

In [ ]:
seasons_gb = da.groupby(da.time.dt.season).mean()
# add the attrs back to the season groupby object
seasons_gb.attrs = da.attrs
# re-order the seasons
seasons_gb = seasons_gb.reindex({"season": ["DJF", "MAM", "JJA", "SON"]})
seasons_gb

Use the seasons groupby object and specify the `season` dimension in the `Facetgrid` call to automatically plot mean backscatter for each season:

In [ ]:
fg_vv = s1_tools.power_to_db(seasons_gb.sel(band="vv")).plot(col="season", cmap=plt.cm.Greys_r);

In [ ]:
fg_vh = s1_tools.power_to_db(seasons_gb.sel(band="vh")).plot(col="season", cmap=plt.cm.Greys_r);

{{conclusion}}

This notebook demonstrated how to access cloud-hosted data from Microsoft Planetary Computer, some basic dataset organization and preliminary exploration and visualization. The following notebook will compare the Planetary Computer dataset to the ASF dataset.

We'll use IPython [storemagic](https://ipython.readthedocs.io/en/stable/config/extensions/storemagic.html) to store the object we created in this notebook, `da`, to use in the comparison. 

In [ ]:
%store da